# Build Complicated WorkFlow - Reflection

Reflection is a prompting strategy used to improve the quality and success rate of agents and similar AI systems. In this exampple, we will build a complex workflow that includes writer agents and critic agents. The writing agents will generate draft solutions, while the critic agent will check mistakes and provide the feedback. This critic-revision process can happen in parallel and in the end one picker agent is employed to pick the best draft.

![FLOW Diagrm](../figs/mermaid-diagram-2024-04-19-021456.png)

Let's begin by specifying the model configuration and examining how's the draft of a single agent.

In [11]:
MODEL_CONFIGURATION = {
    "config_name": "gpt-3.5-turbo",
    "model_type": "openai_chat",
    "model_name": "gpt-3.5-turbo",
    "api_key": "",
    "organizations": "",
    "generate_args": {
        "temperature": 0.5
    }}

In [12]:
from agentscope.agents import DialogAgent
import agentscope
from agentscope.message import Msg
agentscope.init(model_configs=MODEL_CONFIGURATION)
agent = DialogAgent(
    name="writer",
    model_config_name="gpt-3.5-turbo",
    sys_prompt="You are an experience poet. You will receive some critics, try to improve your poem.",
)
INITIAL_PROMPT = """
   Write me a 10-line short poem about the ocean.
"""
msg = Msg(name="user", 
          role="user", 
          content=INITIAL_PROMPT)
res = agent(msg)
print(res.content)

In the vast expanse where sky meets sea,
Whispers of waves sing a melodic plea.
Salt-kissed air dances with seagulls in flight,
As the sun dips below the horizon's light.
Majestic depths hold secrets untold,
Where mysteries lie in the ocean's hold.
Turquoise waters beckon, a siren's call,
A timeless beauty captivating all.
The ocean's embrace, a tranquil repose,
Where the heart finds solace and the spirit glows.




Looks o let's here from the critic agent

In [5]:
CRITIC_PROMPT = (
"You are an poetry critic."
"You appreciate the style of William Shakespeare the most"
"You will receive poem from a writer"
"Reflect and critique the poem. Be severe to maximize improvement.")
critic = DialogAgent(
    name="critic",
    model_config_name="gpt-3.5-turbo",
    sys_prompt=CRITIC_PROMPT,
)

In [6]:
feedback = critic(res)
print(feedback.content)

Your poem captures the serene beauty of the ocean with vivid imagery and a soothing rhythm. However, it lacks the depth and complexity that would truly make it stand out. To enhance your work, try to delve deeper into the emotions and themes you wish to convey. Consider experimenting with different poetic devices such as metaphors, similes, and alliteration to add layers of meaning to your verses. Additionally, pay attention to the structure and form of your poem to create a more impactful and memorable piece. Keep refining your craft to reach the level of mastery seen in the works of Shakespeare.


The critic agent is really being severe indeed.To implement the revision-imporve feedbackloop, we can certainly use some thing like 

```python
from agentscope.pipelines.functional import sequentialpipeline
MAX_RETRIES = 2
for _ in range(MAX_RETRIES):
    workflow = sequentialpipeline(agent, critic)

```

However, we adopt a different approach, which treats the workflow as a graph, as shown in the initial figure of the example. In this approach, each node represents an agent, and every edge represents a function that parses the output of one agent and links it to the next (see *\code\workflow.py*). Based on the simple and intuitive design of AgentScope, you can certainly build complicated systems in the way you pref.

In [10]:
from workflow import SequentialWorkFlow
critic_revision_workflow = SequentialWorkFlow()
critic_revision_workflow.register_contexts({'max_retries': 1, 
                                            'is_solved': False, 
                                            'retry_count': 0, 
                                            'history': []})
writer = DialogAgent(
    name="writer",
    model_config_name="gpt-3.5-turbo",
    sys_prompt="You are a helpful assistant.",
)
critic = DialogAgent(
    name="critic",
    model_config_name="gpt-3.5-turbo",
    sys_prompt=CRITIC_PROMPT,
)
critic_revision_workflow.add_node(writer, "writer")
critic_revision_workflow.add_node(critic, "critic")
critic_revision_workflow.add_node(lambda x: print(x.content), "printer")

def store_draft_callback(msg, workflow, *args, **kwargs):
    workflow.contexts['history'].append(msg.content)
    return msg, 'critic'

def critic_revision_callback(msg, workflow, *args, **kwargs):
    max_count = workflow.contexts['max_retries']
    retry_count = workflow.contexts['retry_count']
    print(f"Retry count: {retry_count}/{max_count}")
    if retry_count >= max_count:
        last_draft = workflow.contexts['history'][-1]
        return Msg(name='writer', content=last_draft), 'EXIT'
    else:
        workflow.contexts['retry_count'] += 1
        return Msg(name='crtic', content=msg.content), 'writer'
    
critic_revision_workflow.add_edge('writer', store_draft_callback)
critic_revision_workflow.add_edge('critic', critic_revision_callback)
critic_revision_workflow.set_entry_node('writer')
critic_revision_workflow.set_exit_node('printer')

In [11]:
msg = Msg(name="user",
            role="user",
            content=INITIAL_PROMPT)
critic_revision_workflow(msg)

Retry count: 0/1
Retry count: 1/1


{'id': '4c8f3a9e970b45bb9acd163111d52158',
 'timestamp': '2024-04-19 03:53:45',
 'name': 'writer',
 'content': "Here is an improved version of the poem:\n\nBeneath the azure sky's expanse,\nThe ocean's song, a hypnotic dance.\nWhispers of waves, a soothing lullaby,\nAs sunlight dances on waves high.\n\nMajestic tides, a timeless flow,\nRevealing secrets of long ago.\nIn depths where mysteries lie unseen,\nA world of wonders, tranquil and serene.\n\nWithin the ocean's vast embrace,\nA symphony of beauty and grace.",
 'role': 'assistant',
 'url': None}

Let's see how the draft get improved 

In [12]:
for i,draft in enumerate(critic_revision_workflow.contexts['history']):
    print("#"*20)
    print(draft)
    print("#"*20)
    print("\u2193") if i < len(critic_revision_workflow.contexts['history'])-1 else None

####################
Beneath the sky so vast and blue,
The ocean dances in shades of hue.
Whispers of waves in rhythmic motion,
A symphony of nature's devotion.

Majestic tides ebb and flow,
Secrets hidden far below.
Mysteries in depths unknown,
A world of wonder to be shown.

In the ocean's embrace, we find peace,
A timeless beauty that will never cease.
####################
↓
####################
Here is an improved version of the poem:

Beneath the azure sky's expanse,
The ocean's song, a hypnotic dance.
Whispers of waves, a soothing lullaby,
As sunlight dances on waves high.

Majestic tides, a timeless flow,
Revealing secrets of long ago.
In depths where mysteries lie unseen,
A world of wonders, tranquil and serene.

Within the ocean's vast embrace,
A symphony of beauty and grace.
####################


Not bad. I personally do prefer the second version and some modifications are impressive 

| Original                             | Improved                              |
|--------------------------------------|---------------------------------------|
| The ocean dances in shades of hue    | The ocean's song, a hypnotic dance    |
| Whispers of waves in rhythmic motion | Whispers of waves, a soothing lullaby |
| A symphony of nature's devotion.     | As sunlight dances on waves high.     |

We can also prompt the critic to adopt different poetic styles, such as those of Emily Dickinson, Robert Frost, or William Shakespeare. This way, the generated drafts will exhibit varying styles, allowing us to ultimately select the one we prefer." In fact you can also let these agents have different models, different prompts, different arguments, sounds like a poem competition among agents. 

In [38]:
workflows = []
poets_names = ["Emily Dickinson", "Robert Frost", "William Shakespeare"]
for i in range(3):
    CRITIC_PROMPT = (
    "You are an poetry critic."
    f"You appreciate the style of {poets_names[i]} the most"
    "You will receive poem about ocean from a writer"
    "Reflect and critique the poem. Be severe to maximize improvement.")
    critic_revision_workflow = SequentialWorkFlow()
    critic_revision_workflow.register_contexts({'max_retries': 1, 
                                                'is_solved': False, 
                                                'retry_count': 0, 
                                                'history': []})
    writer = DialogAgent(
        name="writer",
        model_config_name="gpt-3.5-turbo",
        sys_prompt="You are poet. You will receive some critics, try to improve your poem.",
    )
    critic = DialogAgent(
        name="critic",
        model_config_name="gpt-3.5-turbo",
        sys_prompt=CRITIC_PROMPT,
    )
    critic_revision_workflow.add_node(writer, "writer")
    critic_revision_workflow.add_node(critic, "critic")
    critic_revision_workflow.add_node(lambda x: print(x.content), "printer")
    critic_revision_workflow.add_edge('writer', store_draft_callback)
    critic_revision_workflow.add_edge('critic', critic_revision_callback)
    critic_revision_workflow.set_entry_node('writer')
    critic_revision_workflow.set_exit_node('printer')
    workflows.append(critic_revision_workflow)

Now, we want execute these three workflows in parallel and use another agent to select the best draft.

In [39]:
from workflow import ParallelWorkFlow
def aggregate_fn(results):
    PROMPT = f"Flowing are the drafts"
    for i, result in enumerate(results):
        PROMPT += f"\n\nDraft {i+1}: {result}"
    msg = Msg(name="draft", content=PROMPT)
    return msg

parallel_workflow = ParallelWorkFlow(aggregation_fn=aggregate_fn)
for i, workflow in enumerate(workflows):
    parallel_workflow.add_workflow(f"workflow_{i}", workflow)

large_workflow = SequentialWorkFlow()
large_workflow.add_node(parallel_workflow, "parallel_workflow")

picker_agent = DialogAgent(
    name="picker",
    model_config_name="gpt-3.5-turbo",
    sys_prompt=("You are a poetry editor. "
                "Pick the best draft from the following drafts based on the style and creativity."
                "Just respond with your selected choice and the full draft."
                "Also provide a reason for your choice.")
                )

large_workflow.add_node(picker_agent, "picker")
large_workflow.add_edge("parallel_workflow", "picker")
large_workflow.set_entry_node("parallel_workflow")
large_workflow.set_exit_node("picker")

In [40]:
initial_msg = Msg(name="user", 
                  content="Write me a 10-line short poem about the ocean.")
final_pick = large_workflow(initial_msg)
print(final_pick.content)

Retry count: 0/1
Retry count: 0/1
Retry count: 0/1
Retry count: 1/1
Retry count: 1/1
Retry count: 1/1
I have chosen Draft 3: workflow_2 as the best draft.

Draft 3: workflow_2
In the garden of dreams, where moonbeams dance,
Whispers of stardust in a fleeting trance.
Petals of wishes, blooming bright,
Under the velvet cloak of night.

Reason: This draft stood out to me due to its enchanting imagery and lyrical flow. The use of vivid descriptions like "moonbeams dance" and "whispers of stardust" creates a magical and dreamy atmosphere. The overall elegance and beauty of the language make this draft a standout choice.


---